In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import json
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import os
from datetime import datetime
import cv2
import tempfile
from pathlib import Path

# Load the saved model and class names
try:
    model = tf.keras.models.load_model('cotton_disease_model.keras')
    with open('class_names.json', 'r') as f:
        class_names = json.load(f)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    # Create dummy model and class names for testing if the real ones aren't available
    class_names = ['Healthy', 'Bacterial Blight', 'Fusarium Wilt', 'Verticillium Wilt', 'Root Rot']
    print("⚠️ Using dummy data for testing")

def predict_image(model, image_path, image_size=224):
    """
    Predict the class of an image
    """
    try:
        # Load and preprocess the image
        image = Image.open(image_path).convert('RGB')
        original_image = image.copy()
        image = image.resize((image_size, image_size))
        image_array = np.array(image) / 255.0
        image_array = np.expand_dims(image_array, axis=0)
        
        # Make prediction
        predictions = model.predict(image_array, verbose=0)
        predicted_class_idx = np.argmax(predictions[0])
        predicted_class = class_names[predicted_class_idx]
        confidence = round(100 * np.max(predictions[0]), 2)
        
        return predicted_class, confidence, original_image
    
    except Exception as e:
        print(f"Error predicting image: {e}")
        return None, 0.0, None

def predict_video(model, video_path, image_size=224, num_frames=10):
    """
    Predict the class of a video by analyzing multiple frames
    """
    try:
        # Open the video file
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Select frames to analyze (spread throughout the video)
        frame_indices = np.linspace(0, total_frames-1, min(num_frames, total_frames), dtype=int)
        
        predictions = []
        confidences = []
        frames = []
        
        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            
            if ret:
                # Convert BGR to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_image = Image.fromarray(frame_rgb)
                
                # Resize and preprocess
                image = pil_image.resize((image_size, image_size))
                image_array = np.array(image) / 255.0
                image_array = np.expand_dims(image_array, axis=0)
                
                # Make prediction
                prediction = model.predict(image_array, verbose=0)
                predicted_class_idx = np.argmax(prediction[0])
                predicted_class = class_names[predicted_class_idx]
                confidence = round(100 * np.max(prediction[0]), 2)
                
                predictions.append(predicted_class)
                confidences.append(confidence)
                frames.append(pil_image)
        
        cap.release()
        
        if not predictions:
            return None, 0.0, None, None
        
        # Get the most common prediction
        predicted_class = max(set(predictions), key=predictions.count)
        
        # Calculate average confidence for the most common prediction
        relevant_confidences = [conf for i, conf in enumerate(confidences) if predictions[i] == predicted_class]
        avg_confidence = sum(relevant_confidences) / len(relevant_confidences) if relevant_confidences else 0
        
        # Get a representative frame (the one with highest confidence for the final prediction)
        if relevant_confidences:
            best_frame_idx = confidences.index(max(relevant_confidences))
            representative_frame = frames[best_frame_idx]
        else:
            representative_frame = frames[0] if frames else None
        
        return predicted_class, avg_confidence, representative_frame, frames
    
    except Exception as e:
        print(f"Error predicting video: {e}")
        return None, 0.0, None, None

def get_farmer_advice(disease, confidence, language='both'):
    """
    Get comprehensive advice for farmers based on the detected disease
    """
    advice = {}
    
    # Enhanced disease information database for all 8 diseases
    disease_info = {
        'Aphids': {
            'english': {
                'description': "Aphids are small, soft-bodied insects that suck sap from cotton plants. They cause curling and yellowing of leaves, stunted growth, and produce honeydew that leads to sooty mold growth. Severe infestations can significantly reduce yield.",
                'treatment': "• Imidacloprid (0.05%) or Thiamethoxam (0.01%) spray\n• Neem oil (2-3%) application\n• Release beneficial insects like ladybugs and lacewings\n• Insecticidal soap sprays\n• Remove heavily infested plant parts",
                'prevention': "• Use yellow sticky traps for monitoring\n• Encourage natural predators\n• Avoid excessive nitrogen fertilization\n• Practice crop rotation\n• Remove weed hosts around fields",
                'severity': "Moderate to High (Can cause 20-40% yield loss)",
                'immediate_actions': "Apply insecticide spray, remove heavily infested leaves, introduce biological controls",
                'long_term_management': "Develop integrated pest management strategies, use resistant varieties, maintain ecological balance",
                'economic_impact': "Significant - reduces plant vigor and fiber quality"
            },
            'hindi': {
                'description': "एफिड छोटे, मुलायम शरीर वाले कीट हैं जो कपास के पौधों से रस चूसते हैं। वे पत्तियों के मुड़ने और पीले पड़ने, विकास अवरुद्ध होने और शहद जैसा पदार्थ छोड़ते हैं जिससे काली फफूंदी उगती है। गंभीर संक्रमण उपज को काफी कम कर सकता है।",
                'treatment': "• इमिडाक्लोप्रिड (0.05%) या थायमेथोक्सम (0.01%) स्प्रे\n• नीम तेल (2-3%) अनुप्रयोग\n• लेडीबग और लेसविंग जैसे लाभकारी कीट छोड़ें\n• कीटनाशक साबुन स्प्रे\n• अत्यधिक संक्रमित पौधों के भागों को हटाएं",
                'prevention': "• निगरानी के लिए पीले चिपचिपे जाल का उपयोग करें\n• प्राकृतिक शिकारियों को प्रोत्साहित करें\n• अत्यधिक नाइट्रोजन उर्वरक से बचें\n• फसल चक्र का अभ्यास करें\n• खेतों के आसपास के खरपतवार मेजबानों को हटाएं",
                'severity': "मध्यम से उच्च (20-40% उपज हानि का कारण बन सकता है)",
                'immediate_actions': "कीटनाशक स्प्रे लगाएं, अत्यधिक संक्रमित पत्तियों को हटाएं, जैविक नियंत्रण शुरू करें",
                'long_term_management': "एकीकृत कीट प्रबंधन रणनीतियाँ विकसित करें, प्रतिरोधी किस्मों का उपयोग करें, पारिस्थितिक संतुलन बनाए रखें",
                'economic_impact': "महत्वपूर्ण - पौधे की शक्ति और फाइबर गुणवत्ता कम करता है"
            },
            'marathi': {
                'description': "एफिड्स लहान, मऊ शरीराचे कीटक आहेत जे कापूस वनस्पतींचा रस शोषून घेतात. ते पाने वाकणे आणि पिवळे होणे, वाढ अडकणे आणि मधासारखे पदार्थ सोडतात ज्यामुळे काजळी बुरशी वाढते. गंभीर संसर्ग उत्पादन लक्षणीयरीत्या कमी करू शकतो.",
                'treatment': "• इमिडाक्लोप्रिड (0.05%) किंवा थायमेथोक्सम (0.01%) स्प्रे\n• निंब तेल (2-3%) अर्ज\n• लेडीबग आणि लेसविंग सारख्या फायदेशीर कीटक सोडा\n• कीटकनाशक साबण स्प्रे\n• जास्त प्रमाणात संसर्ग झालेले वनस्पती भाग काढा",
                'prevention': "• निरीक्षणासाठी पिवळे चिकट जाळे वापरा\n• नैसर्गिक शिकारी प्रोत्साहित करा\n• जास्त नायट्रोजन खत टाळा\n• पिक फेरफार सरावा\n• शेतांभोवतीचे तण मेजबान काढा",
                'severity': "मध्यम ते उच्च (20-40% उत्पादन तोटा होऊ शकतो)",
                'immediate_actions': "कीटकनाशक स्प्रे लावा, जास्त प्रमाणात संसर्ग झालेली पाने काढा, जैविक नियंत्रण सुरू करा",
                'long_term_management': "एकात्मिक कीटक व्यवस्थापन धोरणे विकसित करा, प्रतिकारशील किस्म वापरा, पारिस्थितिक संतुलन राखा",
                'economic_impact': "लक्षणीय - वनस्पतीची ताकद आणि फायबर गुणवत्ता कमी करते"
            }
        },
        'Army worm': {
            'english': {
                'description': "Armyworms are caterpillars that can rapidly defoliate cotton plants. They feed on leaves, squares, and bolls, causing extensive damage. They get their name from their behavior of moving in large groups like an army.",
                'treatment': "• Chlorantraniliprole (0.015%) or Emamectin benzoate (0.002%)\n• Spinosad (0.015%) application\n• Biological control with Bacillus thuringiensis\n• Hand picking in early stages\n• Use pheromone traps for monitoring",
                'prevention': "• Regular field monitoring\n• Early detection and intervention\n• Maintain field sanitation\n• Use light traps for adult moths\n• Practice deep plowing to destroy pupae",
                'severity': "High to Very High (Can cause 50-80% yield loss)",
                'immediate_actions': "Apply recommended insecticides immediately, manually remove larvae, use biological controls",
                'long_term_management': "Develop integrated pest management, use resistant varieties, maintain ecological balance",
                'economic_impact': "Severe - can cause complete crop loss if not controlled"
            },
            'hindi': {
                'description': "आर्मीवर्म ऐसे इल्ली हैं जो तेजी से कपास के पौधों की पत्तियों को खा सकते हैं। वे पत्तियों, कलियों और टिंडों को खाते हैं, जिससे व्यापक नुकसान होता है। उन्हें यह नाम बड़े समूहों में सेना की तरह चलने के उनके व्यवहार से मिला है।",
                'treatment': "• क्लोरेंट्रानिलिप्रोल (0.015%) या इमामेक्टिन बेंजोएट (0.002%)\n• स्पिनोसैड (0.015%) अनुप्रयोग\n• बैसिलस थुरिंजिएन्सिस के साथ जैविक नियंत्रण\n• प्रारंभिक अवस्था में हाथ से चुनना\n• निगरानी के लिए फेरोमोन जाल का उपयोग करें",
                'prevention': "• नियमित खेत निगरानी\n• शीघ्र पता लगाना और हस्तक्षेप\n• खेत स्वच्छता बनाए रखें\n• वयस्क पतंगों के लिए प्रकाश जाल का उपयोग करें\n• प्यूपा को नष्ट करने के लिए गहरी जुताई का अभ्यास करें",
                'severity': "उच्च से बहुत उच्च (50-80% उपज हानि का कारण बन सकता है)",
                'immediate_actions': "तुरंत अनुशंसित कीटनाशक लगाएं, लार्वा को हाथ से हटाएं, जैविक नियंत्रण का उपयोग करें",
                'long_term_management': "एकीकृत कीट प्रबंधन विकसित करें, प्रतिरोधी किस्मों का उपयोग करें, पारिस्थितिक संतुलन बनाए रखें",
                'economic_impact': "गंभीर - यदि नियंत्रित नहीं किया गया तो पूरी फसल की हानि हो सकती है"
            },
            'marathi': {
                'description': "आर्मीवर्म हे सुरवंट आहेत जे कापूस वनस्पतींची पाने त्वरीत खाऊ शकतात. ते पाने, कळ्या आणि टिंडे खातात, ज्यामुळे मोठ्या प्रमाणात नुकसान होते. त्यांना हे नाव सेनेसारख्या मोठ्या गटांमध्ये फिरण्याच्या वर्तनामुळे मिळाले आहे.",
                'treatment': "• क्लोरेंट्रानिलिप्रोल (0.015%) किंवा इमामेक्टिन बेंझोएट (0.002%)\n• स्पिनोसॅड (0.015%) अर्ज\n• बॅसिलस थुरिंजिएन्सिस सह जैविक नियंत्रण\n• सुरुवातीच्या टप्प्यात हाताने निवडणे\n• निरीक्षणासाठी फेरोमोन जाळे वापरा",
                'prevention': "• नियमित शेत निरीक्षण\n• लवकर शोध आणि हस्तक्षेप\n• शेत स्वच्छता राखा\n• प्रौढ पतंगांसाठी प्रकाश जाळे वापरा\n• कोश्यांना नष्ट करण्यासाठी खोल नांगरणीचा सराव करा",
                'severity': "उच्च ते अत्यंत उच्च (50-80% उत्पादन तोटा होऊ शकतो)",
                'immediate_actions': "तात्काळ शिफारस केलेले कीटकनाशक लावा, अळ्या हाताने काढा, जैविक नियंत्रण वापरा",
                'long_term_management': "एकात्मिक कीटक व्यवस्थापन विकसित करा, प्रतिकारशील किस्म वापरा, पारिस्थितिक संतुलन राखा",
                'economic_impact': "गंभीर - नियंत्रित न केल्यास संपूर्ण पिक तोटा होऊ शकतो"
            }
        },
        'Bacterial blight': {
            'english': {
                'description': "Bacterial blight is caused by Xanthomonas axonopodis. Characterized by water-soaked lesions that turn brown, angular leaf spots, and blackening of stems. Spreads through rain, irrigation, and contaminated tools.",
                'treatment': "• Copper oxychloride (0.3%) spray at 10-day intervals\n• Streptocycline (500ppm) application\n• Remove and destroy infected plant parts\n• Avoid overhead irrigation",
                'prevention': "• Use certified disease-free seeds\n• 2-3 year crop rotation with non-host crops\n• Avoid working in fields when plants are wet\n• Disinfect tools regularly\n• Plant resistant varieties like LRA 5166",
                'severity': "High (Can cause 20-50% yield loss)",
                'immediate_actions': "Isolate infected plants, apply copper-based bactericide immediately",
                'long_term_management': "Develop resistant varieties, improve field sanitation practices",
                'economic_impact': "Significant - reduced yield quality and quantity"
            },
            'hindi': {
                'description': "बैक्टीरियल ब्लाइट Xanthomonas axonopodis के कारण होता है। पानी से भरे घाव जो भूरे हो जाते हैं, कोणीय पत्ती के धब्बे, और तनों का काला पड़ना इसकी विशेषता है। बारिश, सिंचाई और दूषित उपकरणों के माध्यम से फैलता है।",
                'treatment': "• कॉपर ऑक्सीक्लोराइड (0.3%) 10-दिन के अंतराल पर छिड़काव\n• स्ट्रेप्टोसाइक्लिन (500ppm) अनुप्रयोग\n• संक्रमित पौधों के भागों को हटाएं और नष्ट करें\n• ओवरहेड सिंचाई से बचें",
                'prevention': "• प्रमाणित रोगमुक्त बीजों का उपयोग करें\n• गैर-मेजबान फसलों के साथ 2-3 वर्षीय फसल चक्र\n• पौधों के गीले होने पर खेतों में काम करने से बचें\n• उपकरणों को नियमित रूप से कीटाणुरहित करें\n• LRA 5166 जैसी प्रतिरोधी किस्में लगाएं",
                'severity': "उच्च (20-50% उपज हानि का कारण बन सकता है)",
                'immediate_actions': "संक्रमित पौधों को अलग करें, तुरंत तांबे आधारित जीवाणुनाशक लगाएं",
                'long_term_management': "प्रतिरोधी किस्में विकसित करें, खेत स्वच्छता प्रथाओं में सुधार करें",
                'economic_impact': "महत्वपूर्ण - उपज की गुणवत्ता और मात्रा में कमी"
            },
            'marathi': {
                'description': "बॅक्टेरियल ब्लाइट Xanthomonas axonopodis मुळे होतो. पाण्याने भरलेले घाव जे तपकिरी होतात, कोनीय पानांचे डाग आणि खोड काळे होणे याद्वारे वैशिष्ट्यीकृत. पाऊस, सिंचन आणि दूषित साधनांद्वारे पसरते.",
                'treatment': "• कॉपर ऑक्सिक्लोराईड (0.3%) 10-दिवस अंतराने स्प्रे\n• स्ट्रेप्टोसायक्लिन (500ppm) अर्ज\n• संसर्ग झालेले वनस्पती भाग काढा आणि नष्ट करा\n• ओवरहेड सिंचन टाळा",
                'prevention': "• प्रमाणित रोगमुक्त बियाणे वापरा\n• नॉन-होस्ट पिकसह 2-3 वर्षांचे पीक फेरफार\n• वनस्पती ओल्या असताना शेतात काम करणे टाळा\n• नियमितपणे साधने निर्जंतुक करा\n• LRA 5166 सारख्या प्रतिकारशील किस्म लावा",
                'severity': "उच्च (20-50% उत्पादन तोटा होऊ शकतो)",
                'immediate_actions': "संसर्ग झालेल्या वनस्पती वेगळ्या करा, तात्काळ तांब्यावर आधारित बॅक्टेरिसायड लावा",
                'long_term_management': "प्रतिकारशील किस्म विकसित करा, शेत स्वच्छता पद्धती सुधारा",
                'economic_impact': "लक्षणीय - उत्पादनाची गुणवत्ता आणि प्रमाण कमी"
            }
        },
        'Cotton Boll Rot': {
            'english': {
                'description': "Cotton boll rot is caused by various fungal pathogens including Aspergillus, Fusarium, and Rhizopus species. Characterized by browning, softening, and rotting of cotton bolls. Favored by humid conditions and insect damage.",
                'treatment': "• Carbendazim (0.1%) or Mancozeb (0.25%) spray\n• Remove and destroy infected bolls\n• Improve air circulation\n• Control insect pests that cause boll damage\n• Avoid excessive nitrogen application",
                'prevention': "• Use resistant varieties\n• Practice proper spacing for good air flow\n• Avoid waterlogging\n• Control bollworms and other pests\n• Harvest at proper maturity",
                'severity': "High (Can cause 30-60% yield loss)",
                'immediate_actions': "Remove rotten bolls immediately, apply fungicide spray, improve ventilation",
                'long_term_management': "Develop boll rot resistant varieties, improve field sanitation, practice crop rotation",
                'economic_impact': "Substantial - directly affects lint quality and yield"
            },
            'hindi': {
                'description': "कपास बोल रोट विभिन्न कवक रोगजनकों including Aspergillus, Fusarium, और Rhizopus species के कारण होता है। टिंडों के भूरे होने, नरम होने और सड़ने की विशेषता है। आर्द्र परिस्थितियों और कीट क्षति में फलता-फूलता है।",
                'treatment': "• कार्बेन्डाजिम (0.1%) या मैंकोज़ेब (0.25%) स्प्रे\n• संक्रमित टिंडों को हटाएं और नष्ट करें\n• वायु संचरण में सुधार करें\n• टिंडों को नुकसान पहुंचाने वाले कीटों को नियंत्रित करें\n• अत्यधिक नाइट्रोजन अनुप्रयोग से बचें",
                'prevention': "• प्रतिरोधी किस्मों का उपयोग करें\n• अच्छे वायु प्रवाह के लिए उचित अंतर का अभ्यास करें\n• जलभराव से बचें\n• बोलवर्म और अन्य कीटों को नियंत्रित करें\n• उचित परिपक्वता पर कटाई करें",
                'severity': "उच्च (30-60% उपज हानि का कारण बन सकता है)",
                'immediate_actions': "तुरंत सड़े हुए टिंडों को हटाएं, कवकनाशी स्प्रे लगाएं, वेंटिलेशन में सुधार करें",
                'long_term_management': "बोल रोट प्रतिरोधी किस्में विकसित करें, खेत स्वच्छता में सुधार करें, फसल चक्र का अभ्यास करें",
                'economic_impact': "पर्याप्त - सीधे लिंट की गुणवत्ता और उपज को प्रभावित करता है"
            },
            'marathi': {
                'description': "कापूस बोल रॉट विविध बुरशीजन्य रोगजनकांमुळे होते including Aspergillus, Fusarium, आणि Rhizopus प्रजाती. कापूस टिंड्यांचे तपकिरी होणे, मऊ होणे आणि कुजणे याद्वारे वैशिष्ट्यीकृत. आर्द्र परिस्थिती आणि कीटक नुकसानामध्ये फायद्याचे.",
                'treatment': "• कार्बेंडाजिम (0.1%) किंवा मॅन्कोझेब (0.25%) स्प्रे\n• संसर्ग झालेले टिंडे काढा आणि नष्ट करा\n• वायु संचार सुधारा\n• टिंड्यांना नुकसान पोहोचवणारे कीटक नियंत्रित करा\n• जास्त नायट्रोजन अर्ज टाळा",
                'prevention': "• प्रतिकारशील किस्म वापरा\n• चांगल्या वायु प्रवाहासाठी योग्य अंतर सरावा\n• पाण्याचा सांचा टाळा\n• बोलवर्म आणि इतर कीटक नियंत्रित करा\n• योग्य परिपक्वतेवर कापणी करा",
                'severity': "उच्च (30-60% उत्पादन तोटा होऊ शकतो)",
                'immediate_actions': "तात्काळ कुजलेले टिंडे काढा, फंगिसायड स्प्रे लावा, वायुवीजन सुधारा",
                'long_term_management': "बोल रॉट प्रतिरोधी किस्म विकसित करा, शेत स्वच्छता सुधारा, पिक फेरफार सरावा",
                'economic_impact': "लक्षणीय - थेट लिंटची गुणवत्ता आणि उत्पादनावर परिणाम करते"
            }
        },
        'Green Cotton Boll': {
            'english': {
                'description': "Green cotton boll refers to bolls that remain green and fail to open properly at maturity. This can be caused by various factors including nutrient deficiencies, water stress, or physiological disorders affecting boll development.",
                'treatment': "• Ensure balanced nutrition with proper NPK ratio\n• Maintain adequate soil moisture\n• Apply potassium-rich fertilizers\n• Use plant growth regulators if recommended\n• Harvest at optimal maturity stage",
                'prevention': "• Maintain proper irrigation schedule\n• Use balanced fertilization\n• Plant at recommended density\n• Choose appropriate varieties for the region\n• Monitor soil nutrient levels regularly",
                'severity': "Moderate (Can cause 15-30% yield reduction)",
                'immediate_actions': "Improve irrigation, apply potassium fertilizer, assess nutrient status",
                'long_term_management': "Develop proper irrigation scheduling, optimize fertilization practices, use suitable varieties",
                'economic_impact': "Moderate - affects fiber quality and harvesting efficiency"
            },
            'hindi': {
                'description': "हरा कपास बोल उन टिंडों को संदर्भित करता है जो हरे रहते हैं और परिपक्वता पर ठीक से नहीं खुलते हैं। यह विभिन्न कारकों के कारण हो सकता है including पोषक तत्वों की कमी, पानी का तनाव, या टिंडों के विकास को प्रभावित करने वाले शारीरिक विकार।",
                'treatment': "• उचित NPK अनुपात के साथ संतुलित पोषण सुनिश्चित करें\n• पर्याप्त मिट्टी की नमी बनाए रखें\n• पोटेशियम युक्त उर्वरक लगाएं\n• यदि अनुशंसित हो तो पौध विकास नियामकों का उपयोग करें\n• इष्टतम परिपक्वता अवस्था में कटाई करें",
                'prevention': "• उचित सिंचाई अनुसूची बनाए रखें\n• संतुलित उर्वरक का उपयोग करें\n• अनुशंसित घनत्व पर रोपण करें\n• क्षेत्र के लिए उपयुक्त किस्मों का चयन करें\n• मिट्टी के पोषक तत्वों के स्तर की नियमित निगरानी करें",
                'severity': "मध्यम (15-30% उपज में कमी का कारण बन सकता है)",
                'immediate_actions': "सिंचाई में सुधार करें, पोटेशियम उर्वरक लगाएं, पोषक तत्व स्थिति का आकलन करें",
                'long_term_management': "उचित सिंचाई अनुसूची विकसित करें, उर्वरक प्रथाओं को अनुकूलित करें, उपयुक्त किस्मों का उपयोग करें",
                'economic_impact': "मध्यम - फाइबर की गुणवत्ता और कटाई की दक्षता को प्रभावित करता है"
            },
            'marathi': {
                'description': "हिरवा कापूस बोल हे अशा टिंड्यांचा संदर्भ देते जे हिरवे राहतात आणि परिपक्वतेवर योग्यरित्या उघडत नाहीत. हे विविध घटकांमुळे होऊ शकते including पोषक तुटी, पाणी ताण, किंवा टिंड्यांच्या विकासावर परिणाम करणारे शारीरिक विकार.",
                'treatment': "• योग्य NPK गुणोत्तरासह संतुलित पोषण सुनिश्चित करा\n• पुरेशी माती ओलावा राखा\n• पोटॅशियमयुक्त खते लावा\n• शिफारस केल्यास वनस्पती वाढ नियामक वापरा\n• इष्टतम परिपक्वता टप्प्यावर कापणी करा",
                'prevention': "• योग्य सिंचन वेळापत्रक राखा\n• संतुलित खत वापरा\n• शिफारस केलेल्या घनतेवर लागवड करा\n• प्रदेशासाठी योग्य किस्म निवडा\n• मातीतील पोषकद्रव्य पातळी नियमितपणे निरीक्षण करा",
                'severity': "मध्यम (15-30% उत्पादनात घट होऊ शकते)",
                'immediate_actions': "सिंचन सुधारा, पोटॅशियम खत लावा, पोषकद्रव्य स्थितीचे मूल्यांकन करा",
                'long_term_management': "योग्य सिंचन वेळापत्रक विकसित करा, खत पद्धती ऑप्टिमाइझ करा, योग्य किस्म वापरा",
                'economic_impact': "मध्यम - फायबरची गुणवत्ता आणि कापणी कार्यक्षमता प्रभावित करते"
            }
        },
        'Healthy': {
            'english': {
                'description': "The cotton plant appears healthy with vibrant green leaves, proper growth pattern, and no visible signs of disease or stress.",
                'treatment': "No chemical treatment needed. Continue with regular maintenance and monitoring.",
                'prevention': "• Maintain proper plant spacing (30-45cm)\n• Ensure good drainage in fields\n• Practice 3-year crop rotation with cereals\n• Use balanced NPK fertilizers\n• Regular field sanitation",
                'severity': "None",
                'immediate_actions': "Continue current practices, monitor weekly for any changes",
                'long_term_management': "Maintain soil health with organic matter, practice integrated pest management",
                'economic_impact': "Minimal - healthy plants yield optimal production"
            },
            'hindi': {
                'description': "कपास का पौधा स्वस्थ दिखाई दे रहा है, हरे भरे पत्ते, उचित विकास पैटर्न और बीमारी या तनाव के कोई दृश्य संकेत नहीं हैं।",
                'treatment': "कीटनाशक की आवश्यकता नहीं। नियमित रखरखाव और निगरानी जारी रखें।",
                'prevention': "• उचित पौध spacing (30-45cm) बनाए रखें\n• खेतों में अच्छे जल निकासी सुनिश्चित करें\n• अनाज के साथ 3-वर्षीय फसल चक्र का अभ्यास करें\n• संतुलित NPK उर्वरकों का उपयोग करें\n• नियमित खेत स्वच्छता",
                'severity': "कोई नहीं",
                'immediate_actions': "वर्तमान प्रथाओं को जारी रखें, किसी भी बदलाव के लिए साप्ताहिक निगरानी करें",
                'long_term_management': "जैविक पदार्थों के साथ मिट्टी के स्वास्थ्य को बनाए रखें, एकीकृत कीट प्रबंधन का अभ्यास करें",
                'economic_impact': "न्यूनतम - स्वस्थ पौधे इष्टतम उत्पादन देते हैं"
            },
            'marathi': {
                'description': "कापूसाची झाडे निरोगी दिसत आहेत, चैतन्यदायी हिरवी पाने, योग्य वाढीचा नमुना आणि रोग किंवा तणावाची कोणतीही दृश्यमान चिन्हे नाहीत.",
                'treatment': "कीटकनाशकाची गरज नाही. नियमित देखभाल आणि निरीक्षण सुरू ठेवा.",
                'prevention': "• योग्य झाड spacing (30-45cm) राखा\n• शेतात चांगल्या जलनिकासाची खात्री करा\n• धान्यांसह 3-वर्षांचे पीक रोटेशन सरावा\n• संतुलित NPK खते वापरा\n• नियमित शेत स्वच्छता",
                'severity': "काहीही नाही",
                'immediate_actions': "सध्याच्या पद्धती सुरू ठेवा, कोणत्याही बदलांसाठी साप्ताहिक निरीक्षण करा",
                'long_term_management': "सेंद्रिय पदार्थांसह मातीचे आरोग्य राखा, एकात्मिक कीटक व्यवस्थापनाचा सराव करा",
                'economic_impact': "किमान - निरोगी वनस्पती इष्टतम उत्पादन देतात"
            }
        },
        'Powdery mildew': {
            'english': {
                'description': "Powdery mildew is a fungal disease caused by Erysiphe cichoracearum. Characterized by white powdery spots on leaves and stems, which can spread to cover the entire plant. Favored by moderate temperatures and high humidity.",
                'treatment': "• Sulfur-based fungicides (0.2% solution)\n• Potassium bicarbonate sprays\n• Neem oil application (2-3% solution)\n• Remove severely infected leaves\n• Improve air circulation around plants",
                'prevention': "• Avoid overhead irrigation\n• Maintain proper plant spacing\n• Use resistant varieties\n• Apply preventive fungicides before flowering\n• Remove crop debris after harvest",
                'severity': "Moderate to High (Can cause 10-40% yield loss)",
                'immediate_actions': "Apply sulfur spray immediately, improve ventilation, remove infected plant parts",
                'long_term_management': "Develop resistant varieties, improve field sanitation, practice crop rotation",
                'economic_impact': "Moderate - affects fiber quality and reduces yield"
            },
            'hindi': {
                'description': "पाउडरी मिल्ड्यू Erysiphe cichoracearum नामक कवक के कारण होने वाला एक रोग है। पत्तियों और तनों पर सफेद पाउडर जैसे धब्बों की विशेषता है, जो पूरे पौधे को ढक सकते हैं। मध्यम तापमान और उच्च आर्द्रता में फलता-फूलता है।",
                'treatment': "• सल्फर आधारित कवकनाशी (0.2% घोल)\n• पोटेशियम बाइकार्बोनेट स्प्रे\n• नीम तेल का अनुप्रयोग (2-3% घोल)\n• गंभीर रूप से संक्रमित पत्तियों को हटाएं\n• पौधों के आसपास वायु संचरण में सुधार करें",
                'prevention': "• ओवरहेड सिंचाई से बचें\n• उचित पौध spacing बनाए रखें\n• प्रतिरोधी किस्मों का उपयोग करें\n• फूल आने से पहले निवारक कवकनाशी लगाएं\n• फसल कटाई के बाद अवशेषों को हटाएं",
                'severity': "मध्यम से उच्च (10-40% उपज हानि का कारण बन सकता है)",
                'immediate_actions': "तुरंत सल्फर स्प्रे लगाएं, वेंटिलेशन में सुधार करें, संक्रमित पौधों के भागों को हटाएं",
                'long_term_management': "प्रतिरोधी किस्में विकसित करें, खेत स्वच्छता में सुधार करें, फसल चक्र का अभ्यास करें",
                'economic_impact': "मध्यम - फाइबर की गुणवत्ता को प्रभावित करता है और उपज कम करता है"
            },
            'marathi': {
                'description': "पावडरी मिल्ड्यू हा Erysiphe cichoracearum नावाच्या बुरशीमुळे होणारा रोग आहे. पाने आणि खोडांवर पांढरे पावडरसारखे डाग याद्वारे वैशिष्ट्यीकृत, जे संपूर्ण वनस्पती झाकू शकतात. मध्यम तापमान आणि उच्च आर्द्रतेत फायद्याचे.",
                'treatment': "• सल्फर-आधारित फंगिसायड्स (0.2% द्रावण)\n• पोटॅशियम बायकार्बोनेट स्प्रे\n• निंब तेल अर्ज (2-3% द्रावण)\n• गंभीरपणे संसर्ग झालेली पाने काढा\n• वनस्पतींच्या आजूबाजूचे वायु संचार सुधारा",
                'prevention': "• ओवरहेड सिंचन टाळा\n• योग्य वनस्पती अंतर राखा\n• प्रतिकारशील किस्म वापरा\n• फुलोरा येण्यापूर्वी प्रति�बंधात्मक फंगिसायड लावा\n• कापणीनंतर पिकाचे अवशेष काढा",
                'severity': "मध्यम ते उच्च (10-40% उत्पादन तोटा होऊ शकतो)",
                'immediate_actions': "तात्काळ सल्फर स्प्रे लावा, वायुवीजन सुधारा, संसर्ग झालेले वनस्पती भाग काढा",
                'long_term_management': "प्रतिकारशील किस्म विकसित करा, शेत स्वच्छता सुधारा, पिक फेरफार सरावा",
                'economic_impact': "मध्यम - फायबरची गुणवत्ता प्रभावित करते आणि उत्पादन कमी करते"
            }
        },
        'Target spot': {
            'english': {
                'description': "Target spot is caused by the fungus Corynespora cassiicola. Characterized by circular, target-like spots with concentric rings on leaves. The spots have dark margins and light centers, eventually causing defoliation.",
                'treatment': "• Chlorothalonil (0.2%) or Azoxystrobin (0.1%) spray\n• Mancozeb (0.25%) application\n• Remove severely infected leaves\n• Improve air circulation\n• Avoid overhead irrigation",
                'prevention': "• Use disease-free seeds\n• Practice crop rotation with non-host crops\n• Maintain proper plant spacing\n• Avoid working in wet fields\n• Remove crop debris after harvest",
                'severity': "Moderate to High (Can cause 20-50% yield loss)",
                'immediate_actions': "Apply fungicide spray, remove infected leaves, improve field ventilation",
                'long_term_management': "Develop resistant varieties, improve field sanitation, practice proper crop rotation",
                'economic_impact': "Significant - reduces photosynthetic area and can cause premature defoliation"
            },
            'hindi': {
                'description': "टारगेट स्पॉट कवक Corynespora cassiicola के कारण होता है। पत्तियों पर गोलाकार, लक्ष्य जैसे धब्बे जिनमें संकेंद्रित वलय होते हैं, इसकी विशेषता है। धब्बों के काले किनारे और हल्के केंद्र होते हैं, जो अंततः पत्ती झड़ने का कारण बनते हैं।",
                'treatment': "• क्लोरोथालोनिल (0.2%) या एज़ोक्सिस्ट्रोबिन (0.1%) स्प्रे\n• मैंकोज़ेब (0.25%) अनुप्रयोग\n• गंभीर रूप से संक्रमित पत्तियों को हटाएं\n• वायु संचरण में सुधार करें\n• ओवरहेड सिंचाई से बचें",
                'prevention': "• रोगमुक्त बीजों का उपयोग करें\n• गैर-मेजबान फसलों के साथ फसल चक्र का अभ्यास करें\n• उचित पौध spacing बनाए रखें\n• गीले खेतों में काम करने से बचें\n• फसल कटाई के बाद अवशेषों को हटाएं",
                'severity': "मध्यम से उच्च (20-50% उपज हानि का कारण बन सकता है)",
                'immediate_actions': "कवकनाशी स्प्रे लगाएं, संक्रमित पत्तियों को हटाएं, खेत वेंटिलेशन में सुधार करें",
                'long_term_management': "प्रतिरोधी किस्में विकसित करें, खेत स्वच्छता में सुधार करें, उचित फसल चक्र का अभ्यास करें",
                'economic_impact': "महत्वपूर्ण - प्रकाश संश्लेषक क्षेत्र को कम करता है और समय से पहले पत्ती झड़ने का कारण बन सकता है"
            },
            'marathi': {
                'description': "टार्गेट स्पॉट बुरशी Corynespora cassiicola मुळे होतो. पानांवर गोलाकार, लक्ष्यासारखे डाग ज्यात केंद्रित रिंग असतात याद्वारे वैशिष्ट्यीकृत. डागांच्या काळ्या किनाऱ्या आणि हलके केंद्र असतात, शेवटी पानगळ होते.",
                'treatment': "• क्लोरोथालोनिल (0.2%) किंवा अझोक्सिस्ट्रोबिन (0.1%) स्प्रे\n• मॅन्कोझेब (0.25%) अर्ज\n• गंभीरपणे संसर्ग झालेली पाने काढा\n• वायु संचार सुधारा\n• ओवरहेड सिंचन टाळा",
                'prevention': "• रोगमुक्त बियाणे वापरा\n• नॉन-होस्ट पिकसह पिक फेरफार सरावा\n• योग्य वनस्पती अंतर राखा\n• ओले शेतात काम करणे टाळा\n• कापणीनंतर पिकाचे अवशेष काढा",
                'severity': "मध्यम ते उच्च (20-50% उत्पादन तोटा होऊ शकतो)",
                'immediate_actions': "फंगिसायड स्प्रे लावा, संसर्ग झालेली पाने काढा, शेत वायुवीजन सुधारा",
                'long_term_management': "प्रतिकारशील किस्म विकसित करा, शेत स्वच्छता सुधारा, योग्य पिक फेरफार सरावा",
                'economic_impact': "लक्षणीय - प्रकाशसंश्लेषक क्षेत्र कमी करते आणि अकाली पानगळ होऊ शकते"
            }
        }
    }
    
    # Get information for the specific disease or use default if not found
    if disease in disease_info:
        disease_data = disease_info[disease]
    else:
        # Default response for unknown diseases
        disease_data = {
            'english': {
                'description': f"Information about {disease} is being updated in our database. This appears to be an uncommon or newly identified cotton disease.",
                'treatment': "Consult with local agricultural extension officers or plant pathologists for appropriate treatment recommendations.",
                'prevention': "General prevention methods include crop rotation, maintaining plant health, proper spacing, and field sanitation practices.",
                'severity': "Unknown - requires expert assessment",
                'immediate_actions': "Isolate affected plants, take clear photos, and consult agricultural experts",
                'long_term_management': "Implement comprehensive integrated pest management strategies",
                'economic_impact': "Unknown - depends on disease severity and spread"
            },
            'hindi': {
                'description': f"{disease} के बारे में जानकारी हमारे डेटाबेस में अपडेट की जा रही है। यह एक असामान्य या नव पहचानी गई कपास बीमारी प्रतीत होती है।",
                'treatment': "उपयुक्त उपचार सिफारिशों के लिए स्थानीय कृषि विस्तार अधिकारियों या पादप रोगविज्ञानियों से परामर्श करें।",
                'prevention': "सामान्य रोकथाम विधियों में फसल चक्र, पौधों के स्वास्थ्य को बनाए रखना, उचित अंतर और खेत स्वच्छता प्रथाएं शामिल हैं।",
                'severity': "अज्ञात - विशेषज्ञ मूल्यांकन की आवश्यकता है",
                'immediate_actions': "प्रभावित पौधों को अलग करें, स्पष्ट तस्वीरें लें और कृषि विशेषज्ञों से परामर्श करें",
                'long_term_management': "व्यापक एकीकृत कीट प्रबंधन रणनीतियों को लागू करें",
                'economic_impact': "अज्ञात - बीमारी की गंभीरता और प्रसार पर निर्भर करता है"
            },
            'marathi': {
                'description': f"{disease} बद्दल माहिती आमच्या डेटाबेसमध्ये अद्ययावत केली जात आहे. हा एक असामान्य किंवा नवीन ओळखलेला कापूस रोग दिसतो.",
                'treatment': "योग्य उपचार शिफारसींसाठी स्थानिक कृषी विस्तार अधिकाऱ्यांकडे किंवा वनस्पती रोगतज्ज्ञांकडे सल्लामसलत करा.",
                'prevention': "सामान्य प्रतिबंध पद्धतींमध्ये पिक फेरफार, वनस्पतींचे आरोग्य राखणे, योग्य अंतर आणि शेत स्वच्छता पद्धती यांचा समावेश होतो.",
                'severity': "अज्ञात - तज्ञ मूल्यांकन आवश्यक आहे",
                'immediate_actions': "प्रभावित झाडे वेगळ्या करा, स्पष्ट फोटो घ्या आणि कृषी तज्ञांशी सल्लामसलत करा",
                'long_term_management': "व्यापक एकात्मिक कीटक व्यवस्थापन धोरणे अंमलात आणा",
                'economic_impact': "अज्ञात - रोगाच्या तीव्रता आणि प्रसारावर अवलंबून आहे"
            }
        }
    
    if language in ['english', 'both']:
        advice['english'] = {
            'disease': disease,
            'confidence': f"{confidence}%",
            'description': disease_data['english']['description'],
            'treatment': disease_data['english']['treatment'],
            'prevention': disease_data['english']['prevention'],
            'severity': disease_data['english']['severity'],
            'immediate_actions': disease_data['english']['immediate_actions'],
            'long_term_management': disease_data['english']['long_term_management'],
            'economic_impact': disease_data['english']['economic_impact'],
        }
    
    if language in ['hindi', 'both']:
        advice['hindi'] = {
            'disease': disease,
            'confidence': f"{confidence}%",
            'description': disease_data['hindi']['description'],
            'treatment': disease_data['hindi']['treatment'],
            'prevention': disease_data['hindi']['prevention'],
            'severity': disease_data['hindi']['severity'],
            'immediate_actions': disease_data['hindi']['immediate_actions'],
            'long_term_management': disease_data['hindi']['long_term_management'],
            'economic_impact': disease_data['hindi']['economic_impact'],
        }
    
    if language in ['marathi', 'both']:
        advice['marathi'] = {
            'disease': disease,
            'confidence': f"{confidence}%",
            'description': disease_data['marathi']['description'],
            'treatment': disease_data['marathi']['treatment'],
            'prevention': disease_data['marathi']['prevention'],
            'severity': disease_data['marathi']['severity'],
            'immediate_actions': disease_data['marathi']['immediate_actions'],
            'long_term_management': disease_data['marathi']['long_term_management'],
            'economic_impact': disease_data['marathi']['economic_impact'],
        }
    
    return advice
    
class ModernCottonDiseaseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🌿 Smart Cotton Doctor")
        self.root.geometry("1200x800")
        self.root.configure(bg='#f8f9fa')
        self.root.minsize(1000, 700)
        
        # Variables
        self.image_path = None
        self.video_path = None
        self.photo = None
        self.prediction_data = None
        self.is_video = False
        self.camera_active = False
        self.cap = None
        
        # Style configuration
        self.setup_styles()
        
        # Create main container
        self.main_container = ttk.Frame(root, padding="0")
        self.main_container.pack(fill=tk.BOTH, expand=True)
        
        # Header
        self.create_header()
        
        # Main content area
        self.create_main_content()
        
        # Footer
        self.create_footer()
        
    def setup_styles(self):
        """Configure modern styles"""
        style = ttk.Style()
        style.theme_use('clam')
        
        # Configure styles
        style.configure('Header.TFrame', background='#2e7d32')
        style.configure('Header.TLabel', background='#2e7d32', foreground='white', font=('Arial', 18, 'bold'))
        style.configure('Accent.TButton', background='#4caf50', foreground='white', font=('Arial', 12, 'bold'))
        style.configure('Secondary.TButton', background='#ff9800', foreground='white', font=('Arial', 11))
        style.configure('Camera.TButton', background='#2196f3', foreground='white', font=('Arial', 11, 'bold'))
        style.configure('Card.TFrame', background='white', relief='raised', borderwidth=2)
        style.configure('Result.TFrame', background='#e8f5e8')
        style.configure('Title.TLabel', font=('Arial', 14, 'bold'), foreground='#2e7d32')
        style.configure('Subtitle.TLabel', font=('Arial', 12, 'bold'), foreground='#555555')
        style.configure('Normal.TLabel', font=('Arial', 11), foreground='#333333')
        style.configure('Footer.TFrame', background='#e0e0e0')
        
    def create_header(self):
        """Create application header"""
        header_frame = ttk.Frame(self.main_container, style='Header.TFrame', height=80)
        header_frame.pack(fill=tk.X, pady=(0, 20))
        header_frame.pack_propagate(False)
        
        # Title
        title_label = ttk.Label(header_frame, text="🌿 SMART COTTON DOCTOR", style='Header.TLabel')
        title_label.pack(side=tk.LEFT, padx=20, pady=20)
        
        # Subtitle
        subtitle_label = ttk.Label(header_frame, text="AI-Powered Cotton Disease Detection", 
                                  background='#2e7d32', foreground='#e8f5e8', font=('Arial', 10))
        subtitle_label.pack(side=tk.LEFT, padx=(0, 20), pady=20)
        
        # Current time
        self.time_label = ttk.Label(header_frame, text="", background='#2e7d32', foreground='#e8f5e8', font=('Arial', 9))
        self.time_label.pack(side=tk.RIGHT, padx=20, pady=20)
        self.update_time()
        
    def create_main_content(self):
        """Create main content area"""
        main_content = ttk.Frame(self.main_container)
        main_content.pack(fill=tk.BOTH, expand=True, padx=20, pady=(0, 20))
        
        # Left panel - Image upload and preview
        left_panel = ttk.Frame(main_content, width=400)
        left_panel.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 20))
        left_panel.pack_propagate(False)
        
        # Right panel - Results
        right_panel = ttk.Frame(main_content)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Left panel content
        self.create_left_panel(left_panel)
        
        # Right panel content
        self.create_right_panel(right_panel)
        
    def create_left_panel(self, parent):
        """Create left panel with image upload"""
        # Upload card
        upload_card = ttk.Frame(parent, style='Card.TFrame', padding=20)
        upload_card.pack(fill=tk.X, pady=(0, 20))
        
        ttk.Label(upload_card, text="📷 CAPTURE OR UPLOAD MEDIA", style='Title.TLabel').pack(anchor=tk.W, pady=(0, 15))
        
        # Upload buttons frame
        upload_buttons_frame = ttk.Frame(upload_card)
        upload_buttons_frame.pack(fill=tk.X, pady=10)
        
        # Camera capture button
        camera_btn = ttk.Button(upload_buttons_frame, text="📷 Use Camera", 
                               command=self.start_camera, style='Camera.TButton')
        camera_btn.pack(side=tk.LEFT, padx=(0, 10))
        
        # Image upload button
        image_btn = ttk.Button(upload_buttons_frame, text="📁 Select Image", 
                               command=lambda: self.upload_media('image'), style='Accent.TButton')
        image_btn.pack(side=tk.LEFT, padx=(0, 10))
        
        # Video upload button
        video_btn = ttk.Button(upload_buttons_frame, text="🎥 Select Video", 
                               command=lambda: self.upload_media('video'), style='Accent.TButton')
        video_btn.pack(side=tk.LEFT)
        
        # Supported formats
        formats_label = ttk.Label(upload_card, text="Supported: JPG, JPEG, PNG, BMP, MP4, AVI, MOV", 
                                 style='Normal.TLabel', foreground='#666666')
        formats_label.pack(pady=5)
        
        # Media type indicator
        self.media_type_label = ttk.Label(upload_card, text="No media selected", 
                                         style='Normal.TLabel', foreground='#2e7d32')
        self.media_type_label.pack(pady=5)
        
        # Media preview frame
        preview_frame = ttk.Frame(parent, style='Card.TFrame', padding=15)
        preview_frame.pack(fill=tk.BOTH, expand=True)
        
        ttk.Label(preview_frame, text="🖼️ MEDIA PREVIEW", style='Subtitle.TLabel').pack(anchor=tk.W, pady=(0, 10))
        
        self.preview_label = ttk.Label(preview_frame, text="No media selected\n\nSelect an image or video to begin", 
                                      background='#f8f9fa', foreground='#666666', 
                                      font=('Arial', 10), justify=tk.CENTER)
        self.preview_label.pack(fill=tk.BOTH, expand=True)
        
        # Quick actions frame
        actions_frame = ttk.Frame(parent, style='Card.TFrame', padding=15)
        actions_frame.pack(fill=tk.X, pady=(20, 0))
        
        ttk.Label(actions_frame, text="⚡ QUICK ACTIONS", style='Subtitle.TLabel').pack(anchor=tk.W, pady=(0, 10))
        
        action_buttons = ttk.Frame(actions_frame)
        action_buttons.pack(fill=tk.X)
        
        ttk.Button(action_buttons, text="🔄 Clear", command=self.clear_all, 
                  style='Secondary.TButton').pack(side=tk.LEFT, padx=(0, 10))
        ttk.Button(action_buttons, text="💾 Save Report", command=self.save_report, 
                  style='Secondary.TButton').pack(side=tk.LEFT)
        
    def create_right_panel(self, parent):
        """Create right panel with results"""
        # Results notebook for tabs
        self.notebook = ttk.Notebook(parent)
        self.notebook.pack(fill=tk.BOTH, expand=True)
        
        # Prediction tab
        prediction_tab = ttk.Frame(self.notebook, padding=15)
        self.notebook.add(prediction_tab, text="🔍 Prediction Results")
        
        # Advice tab
        advice_tab = ttk.Frame(self.notebook, padding=15)
        self.notebook.add(advice_tab, text="🌱 Farmer Advice")
        
        # Details tab
        details_tab = ttk.Frame(self.notebook, padding=15)
        self.notebook.add(details_tab, text="📊 Detailed Analysis")
        
        # Create content for each tab
        self.create_prediction_tab(prediction_tab)
        self.create_advice_tab(advice_tab)
        self.create_details_tab(details_tab)
        
    def create_prediction_tab(self, parent):
        """Create prediction results tab"""
        # Result card
        result_card = ttk.Frame(parent, style='Result.TFrame', padding=20)
        result_card.pack(fill=tk.X, pady=(0, 20))
        
        self.result_title = ttk.Label(result_card, text="Awaiting Analysis...", 
                                     style='Title.TLabel', foreground='#666666')
        self.result_title.pack(anchor=tk.W, pady=(0, 10))
        
        self.confidence_label = ttk.Label(result_card, text="Confidence: -", 
                                         style='Subtitle.TLabel', foreground='#666666')
        self.confidence_label.pack(anchor=tk.W, pady=(0, 15))
        
        # Media type indicator in results
        self.media_type_result = ttk.Label(result_card, text="", 
                                          style='Normal.TLabel', foreground='#666666')
        self.media_type_result.pack(anchor=tk.W, pady=(0, 15))
        
        # Status indicators
        status_frame = ttk.Frame(result_card)
        status_frame.pack(fill=tk.X, pady=(0, 15))
        
        self.status_icon = ttk.Label(status_frame, text="⏳", font=('Arial', 24))
        self.status_icon.pack(side=tk.LEFT, padx=(0, 15))
        
        self.status_text = ttk.Label(status_frame, text="Ready to analyze cotton media", 
                                    style='Normal.TLabel', foreground='#666666')
        self.status_text.pack(side=tk.LEFT)
        
        # Action required frame
        self.action_frame = ttk.Frame(result_card, style='Card.TFrame', padding=15)
        self.action_frame.pack(fill=tk.X)
        self.action_frame.pack_forget()  # Hide initially
        
    def create_advice_tab(self, parent):
        """Create farmer advice tab"""
        # Create notebook for languages
        self.lang_notebook = ttk.Notebook(parent)
        self.lang_notebook.pack(fill=tk.BOTH, expand=True)
        
        # English tab
        english_tab = ttk.Frame(self.lang_notebook, padding=15)
        self.lang_notebook.add(english_tab, text="🇬🇧 English")
        
        # Hindi tab
        hindi_tab = ttk.Frame(self.lang_notebook, padding=15)
        self.lang_notebook.add(hindi_tab, text="🇮🇳 Hindi")
        
        # Marathi tab
        marathi_tab = ttk.Frame(self.lang_notebook, padding=15)
        self.lang_notebook.add(marathi_tab, text="🇮🇳 Marathi")
        
        # Create text widgets for each language
        self.english_text = tk.Text(english_tab, wrap=tk.WORD, font=('Arial', 11), 
                                   bg='white', relief='flat', padx=10, pady=10, height=15)
        self.english_text.pack(fill=tk.BOTH, expand=True)
        
        self.hindi_text = tk.Text(hindi_tab, wrap=tk.WORD, font=('Arial', 11), 
                                 bg='white', relief='flat', padx=10, pady=10, height=15)
        self.hindi_text.pack(fill=tk.BOTH, expand=True)
        
        self.marathi_text = tk.Text(marathi_tab, wrap=tk.WORD, font=('Arial', 11), 
                                   bg='white', relief='flat', padx=10, pady=10, height=15)
        self.marathi_text.pack(fill=tk.BOTH, expand=True)
        
        # Make text widgets read-only
        for text_widget in [self.english_text, self.hindi_text, self.marathi_text]:
            text_widget.config(state=tk.DISABLED)
        
    def create_details_tab(self, parent):
        """Create detailed analysis tab"""
        # Create scrollable frame
        canvas = tk.Canvas(parent, bg='white')
        scrollbar = ttk.Scrollbar(parent, orient=tk.VERTICAL, command=canvas.yview)
        self.scrollable_frame = ttk.Frame(canvas)
        
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
    def create_footer(self):
        """Create application footer"""
        footer_frame = ttk.Frame(self.main_container, style='Footer.TFrame', height=40)
        footer_frame.pack(fill=tk.X, side=tk.BOTTOM)
        footer_frame.pack_propagate(False)
        
        # Footer content
        footer_text = "🌾 Smart Farming Assistant • AI-Powered Crop Protection • v1.0"
        footer_label = ttk.Label(footer_frame, text=footer_text, 
                                background='#e0e0e0', foreground='#666666',
                                font=('Arial', 9))
        footer_label.pack(pady=10)
        
    def update_time(self):
        """Update current time in header"""
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.time_label.config(text=f"🕒 {current_time}")
        self.root.after(1000, self.update_time)
        
    def start_camera(self):
        """Start the camera for capturing images"""
        if self.camera_active:
            self.stop_camera()
            return
            
        try:
            # Release any previous camera instance
            if self.cap is not None:
                self.cap.release()
                
            # Initialize camera
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Camera Error", "Could not access camera. Please check if it's connected and try again.")
                return
                
            self.camera_active = True
            self.media_type_label.config(text="Camera active - Click preview to capture")
            
            # Update button text
            for widget in self.root.winfo_children():
                if isinstance(widget, ttk.Frame):
                    for child in widget.winfo_children():
                        if isinstance(child, ttk.Frame) and "upload" in str(child).lower():
                            for btn in child.winfo_children():
                                if isinstance(btn, ttk.Button) and "Use Camera" in btn.cget("text"):
                                    btn.config(text="📷 Stop Camera")
            
            # Start camera preview
            self.show_camera_preview()
            
        except Exception as e:
            messagebox.showerror("Camera Error", f"Error accessing camera: {str(e)}")
            
    def stop_camera(self):
        """Stop the camera"""
        self.camera_active = False
        if self.cap is not None:
            self.cap.release()
            self.cap = None
            
        # Update button text
        for widget in self.root.winfo_children():
            if isinstance(widget, ttk.Frame):
                for child in widget.winfo_children():
                    if isinstance(child, ttk.Frame) and "upload" in str(child).lower():
                        for btn in child.winfo_children():
                            if isinstance(btn, ttk.Button) and "Stop Camera" in btn.cget("text"):
                                btn.config(text="📷 Use Camera")
        
        self.media_type_label.config(text="Camera stopped")
        self.preview_label.config(image=None, text="No media selected\n\nSelect an image or video to begin")
        
    def show_camera_preview(self):
        """Show camera preview in the preview label"""
        if not self.camera_active or self.cap is None:
            return
            
        ret, frame = self.cap.read()
        if ret:
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame_rgb)
            
            # Resize for preview
            pil_image.thumbnail((350, 350))
            self.photo = ImageTk.PhotoImage(pil_image)
            
            # Update preview label with click event
            self.preview_label.config(image=self.photo, text="Click to capture image")
            self.preview_label.bind('<Button-1>', self.capture_image)
            
        # Continue preview
        if self.camera_active:
            self.root.after(10, self.show_camera_preview)
            
    def capture_image(self, event):
        """Capture image from camera"""
        if not self.camera_active or self.cap is None:
            return
            
        ret, frame = self.cap.read()
        if ret:
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame_rgb)
            
            # Save to temporary file
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
            self.image_path = temp_file.name
            pil_image.save(self.image_path)
            
            # Stop camera
            self.stop_camera()
            
            # Process the image
            self.video_path = None
            self.is_video = False
            self.media_type_label.config(text="Image captured from camera")
            self.show_loading_state()
            self.root.after(100, self.process_media)
            
    def upload_media(self, media_type):
        """Handle media upload (image or video)"""
        if self.camera_active:
            self.stop_camera()
            
        if media_type == 'image':
            file_types = [
                ("Image files", "*.jpg *.jpeg *.png *.bmp *.gif"),
                ("All files", "*.*")
            ]
            title = "Select Cotton Plant Image"
        else:  # video
            file_types = [
                ("Video files", "*.mp4 *.avi *.mov *.mkv *.wmv"),
                ("All files", "*.*")
            ]
            title = "Select Cotton Plant Video"
        
        file_path = filedialog.askopenfilename(title=title, filetypes=file_types)
        
        if file_path:
            if media_type == 'image':
                self.image_path = file_path
                self.video_path = None
                self.is_video = False
                self.media_type_label.config(text="Image selected")
            else:
                self.video_path = file_path
                self.image_path = None
                self.is_video = True
                self.media_type_label.config(text="Video selected")
                
            self.show_loading_state()
            # Process the media after a short delay to allow UI to update
            self.root.after(100, self.process_media)
            
    def show_loading_state(self):
        """Show loading state while processing"""
        self.status_icon.config(text="⏳")
        if self.camera_active:
            media_type = "camera image"
        else:
            media_type = "video" if self.is_video else "image"
        self.status_text.config(text=f"Analyzing {media_type}...")
        self.result_title.config(text="Processing...", foreground='#ff9800')
        self.confidence_label.config(text="Calculating confidence...")
        
        # Display media preview
        try:
            if self.is_video and self.video_path:
                # Extract a frame from the video for preview
                cap = cv2.VideoCapture(self.video_path)
                ret, frame = cap.read()
                if ret:
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    pil_image = Image.fromarray(frame_rgb)
                    pil_image.thumbnail((350, 350))
                    self.photo = ImageTk.PhotoImage(pil_image)
                    self.preview_label.config(image=self.photo, text="")
                cap.release()
            elif self.image_path:
                image = Image.open(self.image_path)
                image.thumbnail((350, 350))
                self.photo = ImageTk.PhotoImage(image)
                self.preview_label.config(image=self.photo, text="")
        except Exception as e:
            print(f"Error loading media preview: {e}")
            self.preview_label.config(text="Error loading media preview")
            
    def process_media(self):
        """Process the uploaded media and make prediction"""
        try:
            if self.is_video:
                # Process video
                predicted_class, confidence, representative_frame, all_frames = predict_video(model, self.video_path)
                
                if predicted_class is None:
                    self.show_error_state("Failed to process the video.")
                    return
                
                # Store prediction data
                self.prediction_data = {
                    'class': predicted_class,
                    'confidence': confidence,
                    'media_type': 'video',
                    'representative_frame': representative_frame,
                    'all_frames': all_frames,
                    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
            else:
                # Process image
                predicted_class, confidence, original_image = predict_image(model, self.image_path)
                
                if predicted_class is None:
                    self.show_error_state("Failed to process the image.")
                    return
                
                # Store prediction data
                self.prediction_data = {
                    'class': predicted_class,
                    'confidence': confidence,
                    'media_type': 'image',
                    'image': original_image,
                    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
            
            # Update UI with results
            self.update_results_display()
            
        except Exception as e:
            print(f"Error in process_media: {e}")
            self.show_error_state(f"An error occurred: {str(e)}")
            
    def update_results_display(self):
        """Update UI with prediction results"""
        if not self.prediction_data:
            return
            
        pred_class = self.prediction_data['class']
        confidence = self.prediction_data['confidence']
        media_type = self.prediction_data['media_type']
        
        # Update prediction tab
        self.result_title.config(text=f"Detected: {pred_class}", foreground='#2e7d32')
        self.confidence_label.config(text=f"Confidence: {confidence}%")
        self.media_type_result.config(text=f"Media type: {media_type.upper()}")
        
        # Update status
        if confidence > 80:
            self.status_icon.config(text="✅")
            self.status_text.config(text="High confidence detection")
        elif confidence > 60:
            self.status_icon.config(text="⚠️")
            self.status_text.config(text="Moderate confidence detection")
        else:
            self.status_icon.config(text="❓")
            self.status_text.config(text="Low confidence - please verify manually")
        
        # Show action required
        self.show_action_required(pred_class)
        
        # Update advice tabs
        self.update_advice_tabs(pred_class, confidence)
        
        # Update details tab
        self.update_details_tab(pred_class, confidence)
        
    def show_action_required(self, disease_class):
        """Show action required based on disease"""
        self.action_frame.pack(fill=tk.X)
        
        for widget in self.action_frame.winfo_children():
            widget.destroy()
            
        if disease_class == 'Healthy':
            ttk.Label(self.action_frame, text="✅ NO ACTION REQUIRED", 
                     style='Title.TLabel', foreground='#4caf50').pack(anchor=tk.W)
            ttk.Label(self.action_frame, text="Your cotton plant appears healthy. Continue regular monitoring.", 
                     style='Normal.TLabel').pack(anchor=tk.W)
        else:
            ttk.Label(self.action_frame, text="🚨 IMMEDIATE ACTION RECOMMENDED", 
                     style='Title.TLabel', foreground='#f44336').pack(anchor=tk.W)
            ttk.Label(self.action_frame, text="Disease detected! Please check the Farmer Advice tab for treatment instructions.", 
                     style='Normal.TLabel').pack(anchor=tk.W)
            
    def update_advice_tabs(self, disease_class, confidence):
        """Update advice tabs with comprehensive information"""
        advice = get_farmer_advice(disease_class, confidence, 'both')
        
        # English advice
        self.english_text.config(state=tk.NORMAL)
        self.english_text.delete(1.0, tk.END)
        english_content = f"""🌱 {disease_class} - Detection Confidence: {confidence}%

📝 DISEASE DESCRIPTION:
{advice['english']['description']}

🚨 IMMEDIATE ACTIONS:
{advice['english']['immediate_actions']}

⚡ RECOMMENDED TREATMENT:
{advice['english']['treatment']}

🛡️ PREVENTION MEASURES:
{advice['english']['prevention']}

📈 LONG-TERM MANAGEMENT:
{advice['english']['long_term_management']}

⚠️ DISEASE SEVERITY: {advice['english']['severity']}

💰 ECONOMIC IMPACT: {advice['english']['economic_impact']}
"""
        self.english_text.insert(1.0, english_content)
        self.english_text.config(state=tk.DISABLED)
        
        # Hindi advice
        self.hindi_text.config(state=tk.NORMAL)
        self.hindi_text.delete(1.0, tk.END)
        hindi_content = f"""🌱 {disease_class} - Detection Confidence: {confidence}%

📝 रोग विवरण:
{advice['hindi']['description']}

🚨 तत्काल कार्रवाई:
{advice['hindi']['immediate_actions']}

⚡ अनुशंसित उपचार:
{advice['hindi']['treatment']}

🛡️ रोकथाम उपाय:
{advice['hindi']['prevention']}

📈 दीर्घकालिक प्रबंधन:
{advice['hindi']['long_term_management']}

⚠️ रोग गंभीरता: {advice['hindi']['severity']}

💰 आर्थिक प्रभाव: {advice['hindi']['economic_impact']}
"""
        self.hindi_text.insert(1.0, hindi_content)
        self.hindi_text.config(state=tk.DISABLED)
        
        # Marathi advice
        self.marathi_text.config(state=tk.NORMAL)
        self.marathi_text.delete(1.0, tk.END)
        marathi_content = f"""🌱 {disease_class} - Detection Confidence: {confidence}%

📝 रोग वर्णन:
{advice['marathi']['description']}

🚨 तात्काळ कृती:
{advice['marathi']['immediate_actions']}

⚡ शिफारस केलेले उपचार:
{advice['marathi']['treatment']}

🛡️ प्रतिबंध उपाय:
{advice['marathi']['prevention']}

📈 दीर्घकालीन व्यवस्थापन:
{advice['marathi']['long_term_management']}

⚠️ रोग तीव्रता: {advice['marathi']['severity']}

💰 आर्थिक परिणाम: {advice['marathi']['economic_impact']}
"""
        self.marathi_text.insert(1.0, marathi_content)
        self.marathi_text.config(state=tk.DISABLED)
    def update_details_tab(self, disease_class, confidence):
        """Update detailed analysis tab with disease-specific information"""
        for widget in self.scrollable_frame.winfo_children():
            widget.destroy()
        
        # Get comprehensive advice for detailed view
        advice = get_farmer_advice(disease_class, confidence, 'english')
        
        # Add detailed analysis content
        ttk.Label(self.scrollable_frame, text=f"Detailed Analysis: {disease_class}", 
                 style='Title.TLabel').pack(anchor=tk.W, pady=(0, 15))
        
        # Disease-specific detailed information
        disease_specific_info = {
            'Aphids': [
                ("📊 Pest Overview", f"""
Pest: Aphids
Scientific Name: Various Aphid species
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Identification Features", """
• Small, soft-bodied insects (1-3mm)
• Green, black, yellow, or pink coloration
• Pear-shaped bodies with long antennae
• Often found on undersides of leaves
• Produce honeydew that attracts ants
"""),
                ("🌡️ Favorable Conditions", """
• Moderate temperatures (20-25°C)
• Low rainfall periods
• High nitrogen fertilization
• Crowded planting conditions
• Presence of ant colonies
"""),
                ("📈 Damage Symptoms", """
• Curling and yellowing of leaves
• Stunted plant growth
• Honeydew secretion
• Sooty mold development
• Reduced boll formation
"""),
                ("⚡ Life Cycle", """
• Rapid reproduction (10-12 days/generation)
• Both sexual and asexual reproduction
• Can produce winged forms for dispersal
• Overwinters on alternative host plants
• Multiple generations per season
""")
            ],
            'Army worm': [
                ("📊 Pest Overview", f"""
Pest: Armyworm
Scientific Name: Spodoptera species
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Identification Features", """
• Caterpillars with smooth bodies
• Green, brown, or black coloration
• Distinct stripes along body length
• Up to 4cm in length when mature
• Nocturnal feeding behavior
"""),
                ("🌡️ Favorable Conditions", """
• Warm temperatures (25-30°C)
• High humidity conditions
• Dense crop canopy
• Weedy field borders
• Previous crop residue
"""),
                ("📈 Damage Symptoms", """
• Skeletonized leaves
• Defoliation of plants
• Feeding on squares and bolls
• Frass (excrement) visible
• Rapid field-wide damage
"""),
                ("⚡ Life Cycle", """
• Complete metamorphosis (egg-larva-pupa-adult)
• 30-40 days life cycle
• Female moths lay 1000+ eggs
• Larvae go through 6 instars
• Pupation in soil
""")
            ],
            'Bacterial blight': [
                ("📊 Disease Overview", f"""
Disease: Bacterial Blight
Pathogen: Xanthomonas axonopodis pv. malvacearum
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Disease Symptoms", """
• Water-soaked angular leaf spots
• Brown necrotic lesions with yellow halos
• Blackening of stems and petioles
• Boll rot and shedding
• Vascular discoloration
"""),
                ("🌡️ Favorable Conditions", """
• Warm temperatures (28-32°C)
• High humidity and rainfall
• Overhead irrigation
• Wind-driven rain spread
• Contaminated equipment
"""),
                ("📊 Disease Cycle", """
• Survives in crop debris and seeds
• Enters through natural openings/wounds
• Spread by rain splash and irrigation
• Secondary spread through field operations
• Can survive in soil for limited periods
"""),
                ("⚠️ Risk Factors", """
• Susceptible varieties
• Continuous cotton cultivation
• Poor field sanitation
• Windy conditions during rain
• Early season infections
""")
            ],
            'Cotton Boll Rot': [
                ("📊 Disease Overview", f"""
Disease: Cotton Boll Rot
Pathogens: Multiple fungal species
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Disease Symptoms", """
• Soft, watery rot of bolls
• Brown discoloration
• White or colored fungal growth
• Premature boll opening
• Lint discoloration and degradation
"""),
                ("🌡️ Favorable Conditions", """
• High humidity (>85%)
• Warm temperatures (25-30°C)
• Extended wet periods
• Insect damage to bolls
• Dense canopy conditions
"""),
                ("📊 Causal Organisms", """
• Aspergillus species
• Fusarium species
• Rhizopus species
• Diplodia species
• Multiple secondary invaders
"""),
                ("⚠️ Risk Factors", """
• Late-season planting
• High plant density
• Poor air circulation
• Insect pest damage
• Excessive nitrogen
""")
            ],
            'Green Cotton Boll': [
                ("📊 Physiological Disorder", f"""
Disorder: Green Cotton Boll
Type: Physiological/Abiotic disorder
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Identifying Features", """
• Bolls remain green at maturity
• Failure to open properly
• Hard, underdeveloped lint
• Reduced fiber quality
• Delayed harvest operations
"""),
                ("🌡️ Contributing Factors", """
• Potassium deficiency
• Water stress during boll development
• Temperature extremes
• Genetic factors in varieties
• Improper plant nutrition
"""),
                ("📊 Impact Assessment", """
• Reduced harvest efficiency
• Lower fiber quality
• Increased harvesting costs
• Delayed next crop planting
• Economic losses from quality reduction
"""),
                ("⚡ Management Strategies", """
• Balanced fertilization program
• Proper irrigation scheduling
• Soil nutrient monitoring
• Variety selection
• Timely harvest operations
""")
            ],
            'Healthy': [
                ("📊 Plant Status", f"""
Status: Healthy Cotton Plant
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🌿 Optimal Characteristics", """
• Vibrant green foliage
• Proper growth pattern
• Good boll development
• No disease symptoms
• No pest damage signs
"""),
                ("📈 Growth Indicators", """
• Regular node development
• Proper flowering pattern
• Good boll retention
• Uniform plant growth
• Healthy root system
"""),
                ("🛡️ Maintenance Requirements", """
• Regular monitoring
• Balanced nutrition
• Proper irrigation
• Weed control
• Pest surveillance
"""),
                ("📊 Yield Potential", """
• Optimal fiber quality
• Maximum yield capacity
• Good lint characteristics
• High market value
• Sustainable production
""")
            ],
            'Powdery mildew': [
                ("📊 Disease Overview", f"""
Disease: Powdery Mildew
Pathogen: Erysiphe cichoracearum
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Disease Symptoms", """
• White powdery fungal growth
• Circular patches on leaves
• Yellowing of affected areas
• Premature leaf drop
• Reduced photosynthesis
"""),
                ("🌡️ Favorable Conditions", """
• Moderate temperatures (20-27°C)
• High humidity (not free water)
• Shaded conditions
• Poor air circulation
• Dense plant canopy
"""),
                ("📊 Disease Cycle", """
• Airborne conidia spread
• Direct penetration of epidermis
• Rapid disease development
• Secondary spread through wind
• Overwinters on alternative hosts
"""),
                ("⚠️ Risk Factors", """
• Susceptible varieties
• Late planting
• High plant density
• Shaded field conditions
• Poor air movement
""")
            ],
            'Target spot': [
                ("📊 Disease Overview", f"""
Disease: Target Spot
Pathogen: Corynespora cassiicola
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Disease Symptoms", """
• Circular, target-like spots
• Concentric rings on lesions
• Dark margins with light centers
• Premature defoliation
• Reduced boll development
"""),
                ("🌡️ Favorable Conditions", """
• Warm temperatures (25-30°C)
• High humidity (>90%)
• Extended leaf wetness
• Moderate rainfall
• Poor air circulation
"""),
                ("📊 Disease Cycle", """
• Survives in crop debris
• Seed-borne transmission possible
• Spread by rain splash and wind
• Secondary spread through equipment
• Can infect multiple host plants
"""),
                ("⚠️ Risk Factors", """
• Continuous cotton cultivation
• Susceptible varieties
• Early canopy closure
• Overhead irrigation
• High plant density
""")
            ]
        }
        
        # Get disease-specific information or use default
        if disease_class in disease_specific_info:
            details_cards = disease_specific_info[disease_class]
        else:
            # Default information for unknown diseases
            details_cards = [
                ("📊 General Overview", f"""
Disease: {disease_class}
Confidence Level: {confidence}%
Severity: {advice['english']['severity']}

{advice['english']['description']}
"""),
                ("🔍 Common Symptoms", """
• Yellowing or discoloration of leaves
• Wilting or drooping appearance
• Stunted growth patterns
• Lesions or spots on plant parts
• Reduced yield and quality
"""),
                ("🌡️ Environmental Factors", """
• Favorable temperature ranges
• Moisture and humidity conditions
• Soil factors affecting disease
• Seasonal prevalence patterns
• Microclimate influences
"""),
                ("📈 Impact Assessment", """
• Potential yield reduction
• Quality degradation effects
• Economic consequences
• Management cost implications
• Long-term field impact
"""),
                ("⚡ Management Principles", """
• Early detection and monitoring
• Integrated management approach
• Cultural control practices
• Chemical intervention when needed
• Preventive measures implementation
""")
            ]
        
        for title, content in details_cards:
            card = ttk.Frame(self.scrollable_frame, style='Card.TFrame', padding=15)
            card.pack(fill=tk.X, pady=(0, 15))
            
            ttk.Label(card, text=title, style='Subtitle.TLabel').pack(anchor=tk.W, pady=(0, 10))
            
            content_label = ttk.Label(card, text=content.strip(), style='Normal.TLabel', 
                                    justify=tk.LEFT)
            content_label.pack(anchor=tk.W)
        
    def show_error_state(self, error_message):
        """Show error state in the UI"""
        self.status_icon.config(text="❌")
        self.status_text.config(text="Error occurred")
        self.result_title.config(text="Processing Failed", foreground='#f44336')
        self.confidence_label.config(text="Please try another file")
        
        # Show error message
        for widget in self.action_frame.winfo_children():
            widget.destroy()
            
        self.action_frame.pack(fill=tk.X)
        ttk.Label(self.action_frame, text=f"Error: {error_message}", 
                 style='Normal.TLabel', foreground='#f44336').pack(anchor=tk.W)
        
    def clear_all(self):
        """Clear all inputs and results"""
        if self.camera_active:
            self.stop_camera()
            
        self.image_path = None
        self.video_path = None
        self.prediction_data = None
        self.photo = None
        self.is_video = False
        
        # Reset UI elements
        self.preview_label.config(image=None, text="No media selected\n\nSelect an image or video to begin")
        self.preview_label.unbind('<Button-1>')
        self.result_title.config(text="Awaiting Analysis...", foreground='#666666')
        self.confidence_label.config(text="Confidence: -")
        self.media_type_result.config(text="")
        self.media_type_label.config(text="No media selected")
        self.status_icon.config(text="⏳")
        self.status_text.config(text="Ready to analyze cotton media")
        self.action_frame.pack_forget()
        
        # Clear advice tabs
        for text_widget in [self.english_text, self.hindi_text, self.marathi_text]:
            text_widget.config(state=tk.NORMAL)
            text_widget.delete(1.0, tk.END)
            text_widget.config(state=tk.DISABLED)
            
        # Clear details tab
        for widget in self.scrollable_frame.winfo_children():
            widget.destroy()
            
    def save_report(self):
        """Save the current analysis as a report"""
        if not self.prediction_data:
            messagebox.showwarning("No Data", "Please analyze media first before saving a report.")
            return
            
        # Ask for save location
        file_path = filedialog.asksaveasfilename(
            title="Save Report",
            defaultextension=".txt",
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
        )
        
        if not file_path:
            return
            
        try:
            # Get advice in English for the report
            advice = get_farmer_advice(self.prediction_data['class'], 
                                     self.prediction_data['confidence'], 
                                     'english')
            
            # Create report content
            if self.is_video:
                media_name = os.path.basename(self.video_path)
            else:
                media_name = os.path.basename(self.image_path) if self.image_path else "Camera Capture"
            
            report_content = f"""SMART COTTON DOCTOR - ANALYSIS REPORT
=============================================

Generated: {self.prediction_data['timestamp']}
Media Type: {self.prediction_data['media_type']}
Media File: {media_name}

DETECTION RESULTS:
------------------
Disease: {self.prediction_data['class']}
Confidence: {self.prediction_data['confidence']}%

DETAILED ANALYSIS:
------------------
Description: {advice['english']['description']}

Recommended Treatment: {advice['english']['treatment']}

Prevention Measures: {advice['english']['prevention']}

Severity: {advice['english']['severity']}

ADDITIONAL NOTES:
-----------------
- This analysis is AI-powered and should be verified by agricultural experts
- Local conditions may affect treatment effectiveness
- Always follow recommended safety guidelines when applying treatments

=============================================
Report generated by Smart Cotton Doctor v1.0
"""
            
            # Save to file
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(report_content)
                
            messagebox.showinfo("Success", f"Report saved successfully!\n\nLocation: {file_path}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save report: {str(e)}")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = ModernCottonDiseaseApp(root)
    root.mainloop()

✅ Model loaded successfully!


In [2]:
import tensorflow as tf
import numpy as np
import os

def convert_keras_to_tflite(keras_model_path, tflite_output_path, quantization='dynamic'):
    """
    Convert .keras model to TensorFlow Lite format
    
    Args:
        keras_model_path: Path to your .keras model file
        tflite_output_path: Where to save the .tflite file
        quantization: 'dynamic', 'float16', or 'int8'
    """
    
    # ==================== UPDATE THESE PATHS ====================
    # CHANGE THIS to your actual .keras model file path
    keras_model_path = "C:/IMPORTANT/tomato disease/training/cotton_disease_model.keras"
    
    # CHANGE THIS to where you want to save the .tflite file
    tflite_output_path = "C:/IMPORTANT/tomato disease/training/cotton_disease_model.tflite"
    # ============================================================
    
    print("Loading Keras model...")
    try:
        # Load the model
        model = tf.keras.models.load_model(keras_model_path)
        print(f"✓ Model loaded successfully from: {keras_model_path}")
        print(f"  Input shape: {model.input_shape}")
        print(f"  Output shape: {model.output_shape}")
        
    except Exception as e:
        print(f"✗ Error loading model: {e}")
        return False

    print(f"\nConverting to TFLite with {quantization} quantization...")
    try:
        # Create converter
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        
        # Set optimization based on quantization type
        if quantization == 'dynamic':
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            
        elif quantization == 'float16':
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.target_spec.supported_types = [tf.float16]
            
        elif quantization == 'int8':
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            # Create representative dataset for int8 quantization
            def representative_dataset():
                for _ in range(100):
                    # Adjust based on your model's input shape
                    data = np.random.rand(1, 224, 224, 3).astype(np.float32)
                    yield [data]
            converter.representative_dataset = representative_dataset
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        
        # Convert model
        tflite_model = converter.convert()
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(tflite_output_path), exist_ok=True)
        
        # Save the converted model
        with open(tflite_output_path, 'wb') as f:
            f.write(tflite_model)
        
        # Get file size
        file_size = os.path.getsize(tflite_output_path) / (1024 * 1024)
        print(f"✓ Conversion successful!")
        print(f"✓ TFLite model saved to: {tflite_output_path}")
        print(f"✓ Model size: {file_size:.2f} MB")
        
        return True
        
    except Exception as e:
        print(f"✗ Conversion failed: {e}")
        return False

def verify_tflite_model(tflite_model_path):
    """Verify the converted TFLite model works correctly"""
    print(f"\nVerifying TFLite model...")
    
    try:
        interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
        interpreter.allocate_tensors()
        
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        print("✓ Model verification successful!")
        print(f"  Input: {input_details[0]['shape']} {input_details[0]['dtype']}")
        print(f"  Output: {output_details[0]['shape']} {output_details[0]['dtype']}")
        
        return True
        
    except Exception as e:
        print(f"✗ Verification failed: {e}")
        return False

# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    
    # ==================== UPDATE THESE PATHS ====================
    # CHANGE THIS to your actual .keras model file path
    KERAS_MODEL_PATH = "C:/IMPORTANT/tomato disease/training/cotton_disease_model.keras"  # ← UPDATE THIS
    
    # CHANGE THIS to where you want to save the .tflite file
    TFLITE_OUTPUT_PATH = "C:/IMPORTANT/tomato disease/training/cotton_disease_model.tflite"  # ← UPDATE THIS
    # ============================================================
    
    # Choose quantization type: 'dynamic', 'float16', or 'int8'
    QUANTIZATION_TYPE = 'dynamic'  # Best for Raspberry Pi
    
    print("=" * 60)
    print("TensorFlow to TensorFlow Lite Converter")
    print("=" * 60)
    
    # Convert the model
    success = convert_keras_to_tflite(
        keras_model_path=KERAS_MODEL_PATH,
        tflite_output_path=TFLITE_OUTPUT_PATH,
        quantization=QUANTIZATION_TYPE
    )
    
    if success:
        # Verify the conversion
        verify_tflite_model(TFLITE_OUTPUT_PATH)
        
        print("\n" + "=" * 60)
        print("CONVERSION COMPLETE!")
        print("=" * 60)
        print(f"Original: {KERAS_MODEL_PATH}")
        print(f"TFLite: {TFLITE_OUTPUT_PATH}")
        print(f"Quantization: {QUANTIZATION_TYPE}")
        
    else:
        print("\nConversion failed. Please check the paths and try again.")

TensorFlow to TensorFlow Lite Converter
Loading Keras model...
✓ Model loaded successfully from: C:/IMPORTANT/tomato disease/training/cotton_disease_model.keras
  Input shape: (None, 224, 224, 3)
  Output shape: (None, 8)

Converting to TFLite with dynamic quantization...
INFO:tensorflow:Assets written to: C:\Users\Basheer\AppData\Local\Temp\tmp0nxi5yah\assets


INFO:tensorflow:Assets written to: C:\Users\Basheer\AppData\Local\Temp\tmp0nxi5yah\assets


Saved artifact at 'C:\Users\Basheer\AppData\Local\Temp\tmp0nxi5yah'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2531179608976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179995856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179994320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179994704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179996048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179995664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179992784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179990480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179992976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2531179991632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  25